## The implementation of the frequent itemsets maining technique using the Whales Optimization Algorithm


In the modeling of frequent itemset mining using the Whale Optimization Algorithm (WOA), you can define the problem and its components systematically. Here’s how you can model the problem and its elements:

### **1. Problem Definition:**

In the context of **frequent itemset mining**, the goal is to identify itemsets (combinations of items) that appear frequently in a transaction dataset. The **support** of an itemset is the measure used to identify how frequent it is in the dataset.

We can formally define the problem as follows:

**Input:**
- A **transaction database** \( D = \{T_1, T_2, ..., T_m\} \), where each \( T_i \) is a set of items in a transaction. Each transaction \( T_i \) contains a subset of items from the itemset \( I = \{I_1, I_2, ..., I_n\} \).
- A **minimum support threshold** \( in [0, 1] \), which defines the minimum frequency required for an itemset to be considered **frequent**.

**Output:**
- A set of **frequent itemsets** \( 2^I \) such that the support of each itemset \( s(I) ), where \( s(I) \) is the support of the itemset \( I \).

---

### **2. The Role of Whale Optimization Algorithm (WOA):**

The Whale Optimization Algorithm (WOA) is a **population-based metaheuristic** inspired by the hunting behavior of humpback whales. It is used to solve optimization problems by iteratively improving the candidate solutions (whales) based on their fitness.

For frequent itemset mining, the **whales** (solutions) represent different **itemsets**, and their **fitness** is evaluated based on the **support** of the itemset in the transaction database. The algorithm uses different mechanisms (shrinking circle, spiral) to explore the search space and converge to the optimal (or near-optimal) solution — the set of frequent itemsets.

---

### **3. Defining the Search Space:**

The search space in this problem is the space of all possible **itemsets**. This is a **combinatorial optimization** problem, and the search space is very large. Let's break it down:

- The **itemset space**  is the set of all possible itemsets, which can be formed by selecting any subset of the total set of items \( I \).
- For a dataset with \( n \) distinct items, the total search space size is \( 2^n \), representing all possible item combinations (including the empty set).
  
The **position** of each whale in the search space corresponds to a binary vector of length \( n \), where each element represents whether an item is included in the current itemset or not. A value of 1 means the item is included, and 0 means it is not.

### **4. Defining the Whale's Position:**

Each whale's position is a binary vector of size \( n \), where each element \( w_i \) represents whether item \( I_i \) is in the current itemset.

- For example, if there are 5 items \( I_1, I_2, I_3, I_4, I_5 \), a whale's position might look like:
  \[
  \text{Position} = [1, 0, 1, 0, 1]
  \]
  This means the whale is considering the itemset \( \{I_1, I_3, I_5\} \).

### **5. Fitness Function:**

The fitness of each whale (solution) corresponds to the **support** of the itemset represented by that whale. The support of an itemset \( I \) is the proportion of transactions in which the itemset appears. The fitness function \( f(\text{position}) \) can be mathematically defined as:


position = Count of transactions containing the itemset Total number of transactions T_i in D : T_i \supseteq I


Where:
- position is a binary vector representing the current itemset.
- \( T_i \) is a transaction in the database \( D \).
- \( I \) is the itemset corresponding to the current whale's position.

The goal is to find itemsets whose support is above the minimum threshold \( \delta \).

### **6. Constraints:**

- **Minimum support constraint**: Only itemsets with support greater than or equal to the threshold \( \delta \) are considered valid frequent itemsets.
  \[
  f(\text{position}) \geq \delta
  \]

- **Size of the itemset**: The itemset cannot contain duplicate items. Each whale's position vector ensures this by using binary encoding (only 0 or 1).

---

### **7. Pseudo Algorithm for Whale Optimization Algorithm (WOA) for Frequent Itemset Mining:**

Here is a high-level pseudocode for applying WOA to frequent itemset mining:

```
Initialize whale population (binary vectors)
Initialize best position and best fitness

For each iteration t = 1 to MaxT:
    For each whale i in the population:
        Evaluate the fitness (support) of the current itemset
        Update the best position if the current itemset has higher support
    End for

    For each whale i in the population:
        Update whale position based on best position using spiral or shrinking circle mechanism
    End for

    If best fitness > min_support:
        Stop algorithm
    End if
End for

Return the best itemset found with support greater than or equal to min_support
```

### **8. Why is Frequent Itemset Mining an NP-Hard Problem?**

Frequent itemset mining is considered **NP-hard** because:

- **Combinatorial Explosion**: The number of possible itemsets grows exponentially with the number of items. For \( n \) items, there are \( 2^n \) possible subsets, and examining all possible subsets requires exponential time.
- **No Polynomial Time Algorithm**: There is no known polynomial-time algorithm to find all frequent itemsets. The best-known algorithms (like Apriori) have worst-case exponential time complexity, though they can be made more efficient with pruning and other techniques.

This makes frequent itemset mining computationally expensive, especially for large datasets with many items.

---

### **Summary:**

- **Whale Optimization Algorithm (WOA)** is a population-based metaheuristic inspired by the hunting behavior of humpback whales. It is used for optimization problems like **frequent itemset mining**.
- The **search space** is the set of all possible itemsets, which is combinatorially large (exponential in the number of items).
- The **fitness function** evaluates the **support** of itemsets (the fraction of transactions containing the itemset).
- **NP-Hard**: The frequent itemset mining problem is NP-hard due to the combinatorial explosion of possible itemsets and the lack of a polynomial-time algorithm to solve it.

Using WOA for frequent itemset mining is appropriate because of its ability to efficiently explore large search spaces and converge to high-quality solutions, even for NP-hard problems.

Import the dataset

In [ ]:
import csv

# Ouvrir le fichier texte en mode lecture
with open("/content/RecordLink.txt", "r") as txt_file:
    lines = txt_file.readlines()

# Ouvrir un nouveau fichier CSV en mode écriture
with open("RecordLink.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)

    # Écrire chaque ligne après avoir séparé les chiffres
    for line in lines:
        row = line.strip().split(" ")  # Supprime les espaces inutiles et découpe par espace
        writer.writerow(row)  # Écrit la ligne dans le fichier CSV

print("Fichier converti avec succès en CSV !")

Fichier converti avec succès en CSV !


##The WOA with 2 Mechanisms

In [ ]:
import numpy as np
import random
import pandas as pd

# Load your dataset from a CSV file
df = pd.read_csv('/content/RecordLink.csv')

# Map each column to a string value
def map_to_string(row):
    mapped_row = []
    for i, value in enumerate(row):
        mapped_row.append(f"{value}")
    return mapped_row

# Apply the mapping to each row
transactions = [set(map_to_string(row)) for row in df.values]

# List of all items
items = [item for line in transactions for item in line]
n_items = len(set(item for row in df.values for item in row))

# Function to calculate the support of an itemset
def calculate_support(itemset, transactions):
    itemset_set = set(itemset)
    count = sum(1 for transaction in transactions if itemset_set.issubset(transaction))
    return count / len(transactions)


# WOA Parameters
MaxT = 200 # Maximum number of iterations
n_whales = 10  # Number of whales (agents)
min_support = 0.005 # Minimum support threshold
w = 0.5  # Inertia weight
a = 2  # Attraction coefficient
c = 1  # Constant for spiral movement
l = 1  # Randomness parameter
p = 0.5  # Probability for switching between shrinking circle and spiral

# Initialize whale positions (random binary vectors representing itemsets)
def initialize_population():
    population = []
    for _ in range(n_whales):
        position = np.random.randint(2, size=n_items)  # Binary vector
        population.append(position)
    return population

# Update whale position based on WOA mechanisms
def update_position(position, best_position, t):
    # Shrinking Circle mechanism
    if random.random() < p:
        A = 2 * a * random.random() - a  # Random number between [-a, a]
        updated_position = position + A * (best_position - position)  # Shrinking circle formula
    # Spiral mechanism
    else:
        distance = np.abs(best_position - position)
        updated_position = position + c * np.sin(distance) * (best_position - position)

    # Map continuous values to binary using sigmoid function
    updated_position = 1 / (1 + np.exp(-updated_position))  # Sigmoid function
    updated_position = (updated_position > 0.5).astype(int)  # Binary decision
    return updated_position

# Main WOA algorithm
def whale_optimization_algorithm(transactions, MaxT=100, n_whales=10, min_support=0.5):
    population = initialize_population()
    # Initialize best_position with the first whale's position
    best_position = population[0]
    best_support = 0

    # Iterate over each generation (t)
    for t in range(MaxT):

        for i, whale in enumerate(population):
            # Calculate the support (fitness) of the current itemset
            itemset = set([items[i] for i in range(n_items) if whale[i] == 1])
            support = calculate_support(itemset, transactions)

            # Update the best solution if this whale's itemset has higher support
            if support > best_support:
                best_support = support
                best_position = whale

        # Update the population
        for i in range(n_whales):
            population[i] = update_position(population[i], best_position, t)

        if support > min_support:
          break

    # Return the best itemset found after all iterations
    best_itemset = set([items[i] for i in range(n_items) if best_position[i] == 1])
    return best_itemset, best_support

# Run the WOA algorithm
best_itemset, best_support = whale_optimization_algorithm(transactions, MaxT=100, min_support=0.5)
print(f"Best Itemset: {best_itemset}")
print(f"Support: {best_support}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Itemset: {'7', '28', '22', '13', '11', '1', '16', '19'}
Support: 0.01064903576559304


##The WOA using one Mechanism

In [ ]:
import numpy as np
import random
import pandas as pd

# Load your dataset from a CSV file
df = pd.read_csv('/content/RecordLink.csv')

# Map each column to a string value
def map_to_string(row):
    mapped_row = []
    for i, value in enumerate(row):
        mapped_row.append(f"{value}")
    return mapped_row

# Apply the mapping to each row
transactions = [set(map_to_string(row)) for row in df.values]

# List of all items
items = [item for line in transactions for item in line]
n_items = len(set(item for row in df.values for item in row))

# Function to calculate the support of an itemset
def calculate_support(itemset, transactions):
    itemset_set = set(itemset)
    count = sum(1 for transaction in transactions if itemset_set.issubset(transaction))
    return count / len(transactions)


# WOA Parameters
MaxT = 200  # Maximum number of iterations
n_whales = 10  # Number of whales (agents)
min_support = 0.005 # Minimum support threshold
w = 0.5  # Inertia weight
c = 1  # Constant for spiral movement

# Initialize whale positions (random binary vectors representing itemsets)
def initialize_population():
    population = []
    for _ in range(n_whales):
        position = np.random.randint(2, size=n_items)  # Binary vector
        population.append(position)
    return population

def update_position(position, best_position, t):
    # use the Spiral mechanism
    distance = np.abs(best_position - position)
    updated_position = position + c * np.sin(distance) * (best_position - position)

    # Map continuous values to binary using sigmoid function
    updated_position = 1 / (1 + np.exp(-updated_position))  # Sigmoid function
    updated_position = (updated_position > 0.5).astype(int)  # Binary decision
    return updated_position


# Main WOA algorithm
def whale_optimization_algorithm(transactions, MaxT=100, n_whales=10, min_support=0.5):
    population = initialize_population()
    # Initialize best_position with the first whale's position
    best_position = population[0]
    best_support = 0

    # Iterate over each generation (t)
    for t in range(MaxT):

        for i, whale in enumerate(population):
            # Calculate the support (fitness) of the current itemset
            itemset = set([items[i] for i in range(n_items) if whale[i] == 1])
            support = calculate_support(itemset, transactions)

            # Update the best solution if this whale's itemset has higher support
            if support > best_support:
                best_support = support
                best_position = whale

        # Update the population
        for i in range(n_whales):
            population[i] = update_position(population[i], best_position, t)

        if support > min_support:
          break

    # Return the best itemset found after all iterations
    best_itemset = set([items[i] for i in range(n_items) if best_position[i] == 1])
    return best_itemset, best_support

# Run the WOA algorithm
best_itemset, best_support = whale_optimization_algorithm(transactions, MaxT=100, min_support=0.5)
print(f"Best Itemset: {best_itemset}")
print(f"Support: {best_support}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Itemset: {'7', '28', '22', '11', '25', '16', '19'}
Support: 0.01026451692149941


##The Apriori Algorithm

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load your dataset from a CSV file
df = pd.read_csv('/content/RecordLink.csv')

# Map each column to a string value
def map_to_string(row):
    mapped_row = []
    for i, value in enumerate(row):
        mapped_row.append(f"{value}")
    return mapped_row


transactions = [set(map_to_string(row)) for row in df.values]

te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
df_transactions = pd.DataFrame(te_ary, columns=te.columns_)

# Run the apriori algorithm with a minimum support of 0.005
frequent_itemsets = apriori(df_transactions, min_support=0.005, use_colnames=True)


print(frequent_itemsets)




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


       support                               itemsets
0     0.331920                                    (0)
1     0.667850                                    (1)
2     0.999402                                   (11)
3     0.045160                                   (12)
4     0.954840                                   (13)
...        ...                                    ...
8786  0.007359  (7, 21, 15, 27, 13, 11, 24, 1, 5, 19)
8787  0.011301  (6, 15, 22, 13, 11, 24, 1, 27, 5, 19)
8788  0.120915  (21, 6, 27, 13, 11, 24, 1, 5, 16, 18)
8789  0.011359  (21, 6, 27, 13, 11, 24, 1, 5, 16, 19)
8790  0.009506  (7, 28, 22, 13, 11, 1, 25, 5, 16, 19)

[8791 rows x 2 columns]
